In [2]:
import os
import json
import time
import uuid
import boto3
import logging
import requests
from datetime import datetime


In [3]:
# Initialize AWS clients
session = boto3.session.Session()
region = session.region_name

s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
redshift_client = boto3.client('redshift-serverless', region_name=region)
redshift_data_client = boto3.client('redshift-data', region_name=region)
iam_client = boto3.client('iam')
secrets_client = boto3.client('secretsmanager')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")

In [4]:
# Generate unique suffix for resource names
current_time = time.time()
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
suffix = f"{timestamp_str}"

print(f"Using suffix: {suffix}")

Using suffix: 1093701


## Step 1: Download Bedrock Knowledge Base Utilities

Lets download the structured knowledge base utility to help with Knowledge Base configuration and creation.


In [5]:
url = "https://raw.githubusercontent.com/aws-samples/amazon-bedrock-samples/main/rag/knowledge-bases/features-examples/utils/structured_knowledge_base.py"
target_path = "utils/structured_knowledge_base.py"
os.makedirs(os.path.dirname(target_path), exist_ok=True)
response = requests.get(url)
with open(target_path, "w") as f:
    f.write(response.text)
print(f"Downloaded structured KB utils to {target_path}")

Downloaded structured KB utils to utils/structured_knowledge_base.py


In [6]:
from utils.structured_knowledge_base import BedrockStructuredKnowledgeBase


## Step 2: Set up Redshift Serverless Infrastructure

Next we will create the necessary Redshift Serverless components: namespace and workgroup. This infrastructure will host our structured data that the Knowledge Base will query.

In [7]:
# Configuration for Redshift resources
REDSHIFT_NAMESPACE = f'sds-ecommerce-{suffix}'
REDSHIFT_WORKGROUP = f'sds-ecommerce-wg-{suffix}'
REDSHIFT_DATABASE = f'sds-ecommerce'
S3_BUCKET = f'sds-ecommerce-redshift-{suffix}'

print(f"Redshift Namespace: {REDSHIFT_NAMESPACE}")
print(f"Redshift Workgroup: {REDSHIFT_WORKGROUP}")
print(f"Database: {REDSHIFT_DATABASE}")
print(f"S3 Bucket: {S3_BUCKET}")

Redshift Namespace: sds-ecommerce-1093701
Redshift Workgroup: sds-ecommerce-wg-1093701
Database: sds-ecommerce
S3 Bucket: sds-ecommerce-redshift-1093701


In [8]:
def create_iam_role_for_redshift():
    """Create IAM role for Redshift to access S3"""
    try:
        # Get account ID
        account_id = sts_client.get_caller_identity()['Account']
        
        # Create IAM role if it doesn't exist
        role_name = f'RedshiftS3AccessRole-{suffix}'
        try:
            role_response = iam_client.get_role(RoleName=role_name)
            print(f'Role {role_name} already exists')
            return f'arn:aws:iam::{account_id}:role/{role_name}'
        except iam_client.exceptions.NoSuchEntityException:
            trust_policy = {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "redshift.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }
            
            iam_client.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(trust_policy)
            )
            
            # Attach necessary policies
            iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
            )
            
            print(f'Created role {role_name}')
            return f'arn:aws:iam::{account_id}:role/{role_name}'
            
    except Exception as e:
        print(f'Error creating IAM role: {str(e)}')
        raise



redshift_role_arn = create_iam_role_for_redshift()
print(f"Redshift IAM Role ARN: {redshift_role_arn}")


Created role RedshiftS3AccessRole-1093701
Redshift IAM Role ARN: arn:aws:iam::533267284022:role/RedshiftS3AccessRole-1093701


In [9]:
def create_redshift_namespace():
    """Create Redshift Serverless namespace"""
    try:
        # Check if namespace already exists
        try:
            response = redshift_client.get_namespace(namespaceName=REDSHIFT_NAMESPACE)
            print(f'Namespace {REDSHIFT_NAMESPACE} already exists')
            return response['namespace']
        except redshift_client.exceptions.ResourceNotFoundException:
            print(f'Creating namespace {REDSHIFT_NAMESPACE}...')
        
        # Create the namespace
        response = redshift_client.create_namespace(
            namespaceName=REDSHIFT_NAMESPACE,
            adminUsername='admin',
            adminUserPassword='TempPassword123!',  # Change this in production
            dbName=REDSHIFT_DATABASE,
            defaultIamRoleArn=redshift_role_arn,
            iamRoles=[redshift_role_arn]
        )
        
        print(f'Created namespace {REDSHIFT_NAMESPACE}')
        
        # Wait for namespace to be available
        print('Waiting for namespace to be available...')
        max_attempts = 30
        for attempt in range(max_attempts):
            try:
                namespace_response = redshift_client.get_namespace(namespaceName=REDSHIFT_NAMESPACE)
                status = namespace_response['namespace']['status']
                if status == 'AVAILABLE':
                    print(f'Namespace {REDSHIFT_NAMESPACE} is now available')
                    return namespace_response['namespace']
                else:
                    print(f'Namespace status: {status}, waiting...')
                    time.sleep(10)
            except Exception as e:
                print(f'Error checking namespace status: {str(e)}, retrying...')
                time.sleep(10)
        
        print('Timeout waiting for namespace, but proceeding...')
        return response['namespace']
        
    except Exception as e:
        print(f'Error creating namespace: {str(e)}')
        raise

# Create namespace
namespace = create_redshift_namespace()


Creating namespace sds-ecommerce-1093701...
Created namespace sds-ecommerce-1093701
Waiting for namespace to be available...
Namespace sds-ecommerce-1093701 is now available


In [10]:
def create_redshift_workgroup():
    """Create Redshift Serverless workgroup"""
    try:
        # Check if workgroup already exists
        try:
            response = redshift_client.get_workgroup(workgroupName=REDSHIFT_WORKGROUP)
            print(f'Workgroup {REDSHIFT_WORKGROUP} already exists')
            return response['workgroup']
        except redshift_client.exceptions.ResourceNotFoundException:
            print(f'Creating workgroup {REDSHIFT_WORKGROUP}...')
        
        # Create the workgroup
        response = redshift_client.create_workgroup(
            workgroupName=REDSHIFT_WORKGROUP,
            namespaceName=REDSHIFT_NAMESPACE,
            baseCapacity=8,  # Minimum base capacity
            enhancedVpcRouting=False,
            publiclyAccessible=True,
            configParameters=[
                {
                    'parameterKey': 'enable_user_activity_logging',
                    'parameterValue': 'true'
                }
            ]
        )
        
        print(f'Created workgroup {REDSHIFT_WORKGROUP}')
        
        # Wait for workgroup to be available
        print('Waiting for workgroup to be available...')
        max_attempts = 30
        for attempt in range(max_attempts):
            try:
                workgroup_response = redshift_client.get_workgroup(workgroupName=REDSHIFT_WORKGROUP)
                status = workgroup_response['workgroup']['status']
                if status == 'AVAILABLE':
                    print(f'Workgroup {REDSHIFT_WORKGROUP} is now available')
                    return workgroup_response['workgroup']
                else:
                    print(f'Workgroup status: {status}, waiting...')
                    time.sleep(10)
            except Exception as e:
                print(f'Error checking workgroup status: {str(e)}, retrying...')
                time.sleep(10)
        
        print('Timeout waiting for workgroup, but proceeding...')
        return response['workgroup']
        
    except Exception as e:
        print(f'Error creating workgroup: {str(e)}')
        raise

# Create workgroup
workgroup = create_redshift_workgroup()
workgroup_arn = workgroup['workgroupArn']
print(f"Workgroup ARN: {workgroup_arn}")


Creating workgroup sds-ecommerce-wg-1093701...
Created workgroup sds-ecommerce-wg-1093701
Waiting for workgroup to be available...
Workgroup status: CREATING, waiting...
Workgroup status: CREATING, waiting...
Workgroup status: CREATING, waiting...
Workgroup status: CREATING, waiting...
Workgroup sds-ecommerce-wg-1093701 is now available
Workgroup ARN: arn:aws:redshift-serverless:us-west-2:533267284022:workgroup/c9976901-a855-493d-b201-fa8bd402b6a4


## Step 3: Create S3 Bucket and Load Sample Data

We will create an S3 bucket to stage our sample e-commerce data before loading it into Redshift tables.

In [11]:
def create_s3_bucket():
    """Create S3 bucket for data staging"""
    try:
        s3_client.head_bucket(Bucket=S3_BUCKET)
        print(f'Bucket {S3_BUCKET} already exists')
    except:
        try:
            if region == 'us-east-1':
                s3_client.create_bucket(Bucket=S3_BUCKET)
            else:
                s3_client.create_bucket(
                    Bucket=S3_BUCKET,
                    CreateBucketConfiguration={'LocationConstraint': region}
                )
            print(f'Created bucket {S3_BUCKET}')
        except Exception as e:
            print(f'Error creating bucket: {str(e)}')
            raise

# Create S3 bucket
create_s3_bucket()

Created bucket sds-ecommerce-redshift-1093701


In [12]:
def upload_sample_data():
    """Upload sample CSV files to S3"""
    data_files = ['orders.csv', 'order_items.csv', 'payments.csv', 'reviews.csv']
    sds_directory = 'sample_data'
    
    print("Uploading sample data files to S3...")
    files_found = 0
    
    for file_name in data_files:
        local_path = os.path.join(sds_directory, file_name)
        if os.path.exists(local_path):
            # Get file size for informational purposes
            file_size = os.path.getsize(local_path)
            file_size_mb = file_size / (1024 * 1024)
            
            s3_client.upload_file(local_path, S3_BUCKET, file_name)
            print(f'Uploaded {file_name} ({file_size_mb:.1f} MB) to S3')
            files_found += 1
        else:
            print(f'Warning: {local_path} not found')
    
    if files_found == len(data_files):
        print(f"\nSuccessfully uploaded all {files_found} data files to S3")
    else:
        print(f"\nOnly {files_found} out of {len(data_files)} files were found and uploaded")

# Upload sample data
upload_sample_data()


Uploading sample data files to S3...
Uploaded orders.csv (1.8 MB) to S3
Uploaded order_items.csv (1.3 MB) to S3
Uploaded payments.csv (0.8 MB) to S3
Uploaded reviews.csv (0.5 MB) to S3

Successfully uploaded all 4 data files to S3


## Step 4: Create Redshift Tables and Load Data

Now we will create the database tables in Redshift and load our sample e-commerce data.

In [13]:
def wait_for_statement(statement_id):
    """Wait for a Redshift Data API statement to complete"""
    max_attempts = 30
    for attempt in range(max_attempts):
        try:
            response = redshift_data_client.describe_statement(Id=statement_id)
            status = response['Status']
            if status == 'FINISHED':
                return response
            elif status == 'FAILED':
                raise Exception(f"Statement failed: {response.get('Error', 'Unknown error')}")
            elif status == 'CANCELLED':
                raise Exception("Statement was cancelled")
            else:
                print(f"Statement status: {status}, waiting...")
                time.sleep(5)
        except Exception as e:
            if 'Statement failed' in str(e) or 'cancelled' in str(e):
                raise
            print(f"Error checking statement status: {str(e)}, retrying...")
            time.sleep(5)
    
    raise Exception("Timeout waiting for statement to complete")

def run_redshift_statement(sql_statement):
    """Execute a SQL statement in Redshift"""
    try:
        response = redshift_data_client.execute_statement(
            WorkgroupName=REDSHIFT_WORKGROUP,
            Database=REDSHIFT_DATABASE,
            Sql=sql_statement
        )
        statement_id = response['Id']
        print(f"Executing statement: {statement_id}")
        result = wait_for_statement(statement_id)
        print(f"Statement completed successfully")
        return result
    except Exception as e:
        print(f"Error executing statement: {str(e)}")
        raise


In [14]:
# Create tables in Redshift
def create_tables():
    """Create all necessary tables in Redshift"""
    
    # Orders table
    orders_sql = """
    CREATE TABLE IF NOT EXISTS orders (
        order_id VARCHAR(255) PRIMARY KEY,
        customer_id VARCHAR(255),
        order_total DECIMAL(10,2),
        order_status VARCHAR(50),
        payment_method VARCHAR(50),
        shipping_address TEXT,
        created_at TIMESTAMP,
        updated_at TIMESTAMP
    );
    """
    
    # Order Items table
    order_items_sql = """
    CREATE TABLE IF NOT EXISTS order_items (
        order_item_id VARCHAR(255) PRIMARY KEY,
        order_id VARCHAR(255),
        product_id VARCHAR(255),
        quantity INTEGER,
        price DECIMAL(10,2)
    );
    """
    
    # Payments table
    payments_sql = """
    CREATE TABLE IF NOT EXISTS payments (
        payment_id VARCHAR(255) PRIMARY KEY,
        order_id VARCHAR(255),
        customer_id VARCHAR(255),
        amount DECIMAL(10,2),
        payment_method VARCHAR(50),
        payment_status VARCHAR(50),
        created_at DATE
    );
    """
    
    # Reviews table
    reviews_sql = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id VARCHAR(255) PRIMARY KEY,
        product_id VARCHAR(255),
        customer_id VARCHAR(255),
        rating INTEGER,
        created_at DATE
    );
    """
    
    tables = {
        'orders': orders_sql,
        'order_items': order_items_sql,
        'payments': payments_sql,
        'reviews': reviews_sql
    }
    
    for table_name, sql in tables.items():
        print(f"Creating table: {table_name}")
        run_redshift_statement(sql)
        print(f"Created table: {table_name}")
        print("-------------")

# Create tables
create_tables()


Creating table: orders
Executing statement: df2461cb-612b-4f97-a27e-b17ba17ef8d5
Statement status: PICKED, waiting...
Statement completed successfully
Created table: orders
-------------
Creating table: order_items
Executing statement: d30eab0a-f92b-4c7d-845b-a1c9c7e85752
Statement status: PICKED, waiting...
Statement completed successfully
Created table: order_items
-------------
Creating table: payments
Executing statement: ee5381a7-1b36-41f0-88c8-4552ff32252d
Statement status: PICKED, waiting...
Statement completed successfully
Created table: payments
-------------
Creating table: reviews
Executing statement: 301426bf-7540-4682-9998-8f9f9900e4b2
Statement status: PICKED, waiting...
Statement completed successfully
Created table: reviews
-------------


In [15]:
# Load data from S3 into Redshift tables
def load_data_from_s3():
    """Load data from S3 CSV files into Redshift tables"""
    
    tables_and_files = {
        'orders': 'orders.csv',
        'order_items': 'order_items.csv',
        'payments': 'payments.csv',
        'reviews': 'reviews.csv'
    }
    
    for table_name, file_name in tables_and_files.items():
        print(f"Loading data into {table_name} from {file_name}")
        
        copy_sql = f"""
        COPY {table_name}
        FROM 's3://{S3_BUCKET}/{file_name}'
        IAM_ROLE '{redshift_role_arn}'
        CSV
        IGNOREHEADER 1
        DELIMITER ','
        REGION '{region}';
        """
        
        try:
            run_redshift_statement(copy_sql)
            print(f"Loaded data into {table_name}")
        except Exception as e:
            print(f"Error loading data into {table_name}: {str(e)}")

# Load data from S3
load_data_from_s3()

Loading data into orders from orders.csv
Executing statement: 0268be68-6928-40fc-a17f-ef1ae762a4c7
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into orders
Loading data into order_items from order_items.csv
Executing statement: 69045895-bf76-4a54-90a8-f585b9f6c170
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into order_items
Loading data into payments from payments.csv
Executing statement: e558b6a6-fd26-465a-96fc-499aaef39713
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into payments
Loading data into reviews from reviews.csv
Executing statement: b4cc4ea2-c16a-4f5b-af5a-0ab863c37118
Statement status: PICKED, waiting...
Statement completed successfully
Loaded data into reviews


## Step 5: Verify Data Load

Let's verify that our data has been loaded correctly by running some sample queries.

## Step 6: Create Bedrock Knowledge Base with Redshift Data Source

Now we'll create the Bedrock Knowledge Base configured to use our Redshift data as a structured data source.


In [16]:
# Configure Knowledge Base parameters
kb_name = f"redshift-structured-kb-{suffix}"
kb_description = "Structured Knowledge Base for e-commerce data queries using Redshift"
generation_model = "anthropic.claude-3-5-haiku-20241022-v1:0"

print(f"Knowledge Base Name: {kb_name}")


Knowledge Base Name: redshift-structured-kb-1093701


Amazon Bedrock Knowledge Bases uses a service role to connect knowledge bases to structured data stores, retrieve data from these data stores, and generate SQL queries based on user queries and the structure of the data stores. There are several access patterns based on if you're using Redshift Serverless vs Redshift Provisioned Cluster. In this notebook, let's use `Secrets Manager + Redshift Serverless WorkGroup` access pattern.

In [17]:
# Create Secrets Manager secret for Redshift authentication
def create_redshift_secret():
    """Create or retrieve Secrets Manager secret for Redshift database authentication"""
    secret_name = f"redshift-credentials-{suffix}"
    
    try:
        # Check if secret already exists
        try:
            response = secrets_client.get_secret_value(SecretId=secret_name)
            print(f"Secret {secret_name} already exists")
            return response['Name'], f"arn:aws:secretsmanager:{region}:{sts_client.get_caller_identity()['Account']}:secret:{secret_name}"
        except secrets_client.exceptions.ResourceNotFoundException:
            print(f"Creating secret {secret_name}...")
        
        # Create the secret with database credentials
        secret_value = {
            "username": "admin",  # This should match the admin user from the namespace
            "password": "TempPassword123!"  # This should match the password from the namespace
        }
        
        response = secrets_client.create_secret(
            Name=secret_name,
            Description="Redshift database credentials for Bedrock Knowledge Base",
            SecretString=json.dumps(secret_value)
        )
        
        secret_arn = response['ARN']
        print(f"Created secret {secret_name}")
        print(f"Secret ARN: {secret_arn}")
        return secret_name, secret_arn
        
    except Exception as e:
        print(f"Error creating secret: {str(e)}")
        raise

# Create the secret
secret_name, secret_arn = create_redshift_secret()
print(f"Using secret: {secret_name}")
print(f"Secret ARN: {secret_arn}")


Creating secret redshift-credentials-1093701...
Created secret redshift-credentials-1093701
Secret ARN: arn:aws:secretsmanager:us-west-2:533267284022:secret:redshift-credentials-1093701-HMByU1
Using secret: redshift-credentials-1093701
Secret ARN: arn:aws:secretsmanager:us-west-2:533267284022:secret:redshift-credentials-1093701-HMByU1


In [18]:
# Configure Knowledge Base parameters for Redshift Serverless with Secrets Manager authentication
kb_config_param = {
    "type": "SQL",
    "sqlKnowledgeBaseConfiguration": {
        "type": "REDSHIFT",
        "redshiftConfiguration": {
            "storageConfigurations": [{
                "type": "REDSHIFT",
                "redshiftConfiguration": {
                    "databaseName": REDSHIFT_DATABASE
                }
            }],
            "queryEngineConfiguration": {
                "type": "SERVERLESS",
                "serverlessConfiguration": {
                    "workgroupArn": workgroup_arn,
                    "authConfiguration": {}
                }
            }
        }
    }
}


In [19]:
# Configure authentication for Secrets Manager
kb_config_param['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['serverlessConfiguration']['authConfiguration']['type'] = "USERNAME_PASSWORD"
kb_config_param['sqlKnowledgeBaseConfiguration']['redshiftConfiguration']['queryEngineConfiguration']['serverlessConfiguration']['authConfiguration']['usernamePasswordSecretArn'] = secret_arn


In [20]:
kb_config_param

{'type': 'SQL',
 'sqlKnowledgeBaseConfiguration': {'type': 'REDSHIFT',
  'redshiftConfiguration': {'storageConfigurations': [{'type': 'REDSHIFT',
     'redshiftConfiguration': {'databaseName': 'sds-ecommerce'}}],
   'queryEngineConfiguration': {'type': 'SERVERLESS',
    'serverlessConfiguration': {'workgroupArn': 'arn:aws:redshift-serverless:us-west-2:533267284022:workgroup/c9976901-a855-493d-b201-fa8bd402b6a4',
     'authConfiguration': {'type': 'USERNAME_PASSWORD',
      'usernamePasswordSecretArn': 'arn:aws:secretsmanager:us-west-2:533267284022:secret:redshift-credentials-1093701-HMByU1'}}}}}}

In [21]:
try:
    structured_kb = BedrockStructuredKnowledgeBase(
        kb_name=kb_name,
        kb_description=kb_description,
        workgroup_arn=workgroup_arn,
        secrets_arn=secret_arn,
        kbConfigParam=kb_config_param,
        generation_model=generation_model,
        suffix=suffix
    )
    
    print("Knowledge Base created successfully!")
    kb_id = structured_kb.get_knowledge_base_id()
    print(f"Knowledge Base ID: {kb_id}")
    
except Exception as e:
    print(f"Error creating Knowledge Base: {str(e)}")
    raise


Step 1 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForStructuredKnowledgeBase_1093701) and Policies
Step 2 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 6, 21, 16, 43, 21, 853520, tzinfo=tzutc()),
  'description': 'Structured Knowledge Base for e-commerce data queries using '
                 'Redshift',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:533267284022:knowledge-base/AZJAPNQSJU',
  'knowledgeBaseConfiguration': { 'sqlKnowledgeBaseConfiguration': { 'redshiftConfiguration': { 'queryEngineConfiguration': { 'serverlessConfiguration': { 'authConfiguration': { 'type': 'USERNAME_PASSWORD',
                                                                                                                                                                                  'usernamePasswordSecretArn': 'arn:aws:secretsmanager:us-west-2:533267284022:secret:redshift-credentials-1093701-HMByU1'},
                                                      

## Step 6: Database Access Configuration for Secrets Manager + Redshift Serverless WorkGroup


For the Secrets Manager + Redshift Serverless WorkGroup access pattern, the credentials stored in Secrets Manager are used to authenticate with the Redshift database. Since we're using the admin user credentials that were set during namespace creation, no additional database user configuration is required.

**Key Benefits of Secrets Manager approach:**
1. **Simplified setup**: No need to create additional database users or configure IAM role mappings
2. **Credential management**: Passwords can be rotated through Secrets Manager
3. **Direct authentication**: Uses standard username/password authentication to the database


In [22]:
# No additional database configuration needed for Secrets Manager authentication
kb_details = structured_kb.knowledge_base
print("✅ Using Secrets Manager authentication - no additional database user setup required")
print(f"Knowledge Base ARN: {kb_details['knowledgeBaseArn']}")
print(f"Using Secret ARN: {secret_arn}")

✅ Using Secrets Manager authentication - no additional database user setup required
Knowledge Base ARN: arn:aws:bedrock:us-west-2:533267284022:knowledge-base/AZJAPNQSJU
Using Secret ARN: arn:aws:secretsmanager:us-west-2:533267284022:secret:redshift-credentials-1093701-HMByU1


In [23]:

# Verify the secret is accessible and contains the correct credentials
print("🔍 Verifying Secrets Manager configuration...")
try:
    secret_response = secrets_client.get_secret_value(SecretId=secret_name)
    secret_data = json.loads(secret_response['SecretString'])
    print(f"✅ Secret verified - contains username: {secret_data.get('username', 'NOT_FOUND')}")
    print("✅ Password field present:", 'password' in secret_data)
except Exception as e:
    print(f"❌ Error accessing secret: {str(e)}")
    raise

🔍 Verifying Secrets Manager configuration...
✅ Secret verified - contains username: admin
✅ Password field present: True


In [24]:
# Note: For Secrets Manager authentication, the admin user already has full access to all tables
# No additional permission grants are needed since we're using the admin credentials
print("✅ Using admin credentials from Secrets Manager - full database access already available")
print("📝 Note: In production, consider creating a dedicated user with minimal required permissions")

✅ Using admin credentials from Secrets Manager - full database access already available
📝 Note: In production, consider creating a dedicated user with minimal required permissions


## Step 7: Start Ingestion Job

Now that the database permissions are properly configured, let's start the ingestion job to sync the data from the Redshift database.

In [25]:
# Wait a bit for the Knowledge Base to be fully ready
time.sleep(60)
structured_kb.start_ingestion_job()

job  started successfully

{ 'dataSourceId': 'Z68EQ1RAY7',
  'ingestionJobId': 'IB6OHTOYUD',
  'knowledgeBaseId': 'AZJAPNQSJU',
  'startedAt': datetime.datetime(2025, 6, 21, 16, 45, 7, 116877, tzinfo=tzutc()),
  'status': 'FAILED',
  'updatedAt': datetime.datetime(2025, 6, 21, 16, 45, 10, 551762, tzinfo=tzutc())}


In [26]:
# Helper functions for querying the Knowledge Base

def query_with_retrieve_and_generate(kb_id, query):
    """Query using retrieve_and_generate API - returns natural language response"""
    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={"text": query},
            retrieveAndGenerateConfiguration={
                "type": "KNOWLEDGE_BASE",
                "knowledgeBaseConfiguration": {
                    'knowledgeBaseId': kb_id,
                    "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{generation_model}",
                    "retrievalConfiguration": {
                        "vectorSearchConfiguration": {
                            "numberOfResults": 5
                        }
                    }
                }
            }
        )
        return response['output']['text']
    except Exception as e:
        return f"Error: {str(e)}"

def generate_sql_query(kb_arn, query):
    """Generate SQL query from natural language"""
    try:
        response = bedrock_agent_runtime_client.generate_query(
            queryGenerationInput={
                "text": query,
                "type": "TEXT"
            },
            transformationConfiguration={
                "mode": "TEXT_TO_SQL",
                "textToSqlConfiguration": {
                    "type": "KNOWLEDGE_BASE",
                    "knowledgeBaseConfiguration": {
                        "knowledgeBaseArn": kb_details['knowledgeBaseArn']
                    }
                }
            }
        )
        
        if response.get('queries') and len(response['queries']) > 0:
            return response['queries'][0]['sql']
        else:
            return "No SQL generated"
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Helper functions defined")

✅ Helper functions defined


In [27]:
# Test queries
test_queries = [
    "How many orders are in the database?",
    "What is the average order total?"
]

print("🧪 Testing Knowledge Base with sample queries")
print("=" * 60)

for i, query in enumerate(test_queries, 1):
    print(f"\n📝 Query {i}: {query}")
    print("-" * 50)
    
    # Get natural language response
    print("🤖 Natural Language Response:")
    nl_response = query_with_retrieve_and_generate(kb_id, query)
    print(f"   {nl_response}")
    
    # Get generated SQL
    print("\n🔧 Generated SQL:")
    sql_query = generate_sql_query(kb_details['knowledgeBaseArn'], query)
    print(f"   {sql_query}")
    
    print("\n" + "="*50)

🧪 Testing Knowledge Base with sample queries

📝 Query 1: How many orders are in the database?
--------------------------------------------------
🤖 Natural Language Response:
   There are 10,000 orders in the database.

🔧 Generated SQL:
   SELECT COUNT(*) AS "total_orders" FROM public.orders;


📝 Query 2: What is the average order total?
--------------------------------------------------
🤖 Natural Language Response:
   The average order total is $507.84.

🔧 Generated SQL:
   SELECT AVG("order_total") AS "average_order_total" FROM public.orders;



In [28]:
import boto3
from pprint import pprint

# ------------------------------------------------------------------
#  Set your knowledge-base Id here (10-char alpha-numeric string)
# ------------------------------------------------------------------
KB_ID = kb_id

bedrock = boto3.client("bedrock-agent")

def get_ingestion_jobs(kb_id: str):
    """
    Prints the detail of every ingestion job that was ever started
    for every data-source attached to the given knowledge-base.
    """

    # 1) find all data-sources for this KB
    sources = bedrock.list_data_sources(
        knowledgeBaseId=kb_id,
        maxResults=100
    )["dataSourceSummaries"]

    if not sources:
        print("No data-sources found for KB:", kb_id)
        return

    for src in sources:
        ds_id = src["dataSourceId"]
        print(f"\n=== Data-source {ds_id} ({src['name']}) ===")

        # 2) list all ingestion jobs (newest first)
        jobs = bedrock.list_ingestion_jobs(
            knowledgeBaseId=kb_id,
            dataSourceId=ds_id,
            sortBy={"attribute": "STARTED_AT", "order": "DESCENDING"},
            maxResults=100
        )["ingestionJobSummaries"]

        if not jobs:
            print("  (no ingestion jobs yet)")
            continue

        # 3) fetch & print the full record for every job found
        for j in jobs:
            job_id = j["ingestionJobId"]
            detail = bedrock.get_ingestion_job(
                knowledgeBaseId=kb_id,
                dataSourceId=ds_id,
                ingestionJobId=job_id
            )["ingestionJob"]

            print(f"\n  Job {job_id}  |  status: {detail['status']}")
            if detail.get("failureReasons"):
                print("     failureReasons:", detail["failureReasons"])
            if detail.get("statistics"):
                print("     statistics:")
                pprint(detail["statistics"], indent=10)

# ------------------------------------------------------------------
#  Run it
# ------------------------------------------------------------------
get_ingestion_jobs(KB_ID)


=== Data-source Z68EQ1RAY7 (redshift-structured-kb-1093701-ds) ===

  Job IB6OHTOYUD  |  status: FAILED


## Clean Up
Please make sure to uncomment and run the below section to delete all the resources

In [29]:
# # Delete resources
# print("===============================Deleteing resources ==============================\n")
structured_kb.delete_kb( delete_iam_roles_and_policies=True)

======== Data source deleted =========
======== Knowledge base deleted =========
======== All IAM roles and policies deleted =========


In [30]:
def cleanup_redshift_environment():
    """
    Delete all Redshift-related resources including workgroup, namespace, S3 bucket, and IAM role.
    Uses the existing variables defined in the notebook.
    """
    import boto3
    import time
    
    # Initialize clients
    session = boto3.session.Session()
    region = session.region_name
    redshift_client = boto3.client('redshift-serverless', region_name=region)
    iam_client = boto3.client('iam')
    s3 = boto3.resource('s3')
    s3_client = boto3.client('s3')
    secrets_client = boto3.client('secretsmanager')
    
    def wait_for_workgroup_deleted(name, poll_interval=10, max_attempts=60):
        """Wait until workgroup is completely deleted"""
        print(f"  Waiting for workgroup {name} to be deleted...")
        attempts = 0
        while attempts < max_attempts:
            try:
                wg = redshift_client.get_workgroup(workgroupName=name)["workgroup"]
                status = wg["status"]
                print(f"    Workgroup status: {status}")
                if status == "DELETED":
                    break
                time.sleep(poll_interval)
                attempts += 1
            except redshift_client.exceptions.ResourceNotFoundException:
                print("    Workgroup deleted successfully")
                return
        
        if attempts >= max_attempts:
            print(f"    Warning: Timeout waiting for workgroup deletion after {max_attempts * poll_interval} seconds")
    
    def wait_for_namespace_deleted(name, poll_interval=10, max_attempts=60):
        """Wait until namespace is completely deleted"""
        print(f"  Waiting for namespace {name} to be deleted...")
        attempts = 0
        while attempts < max_attempts:
            try:
                redshift_client.get_namespace(namespaceName=name)
                print(f"    Namespace still exists, waiting...")
                time.sleep(poll_interval)
                attempts += 1
            except redshift_client.exceptions.ResourceNotFoundException:
                print("    Namespace deleted successfully")
                return
        
        if attempts >= max_attempts:
            print(f"    Warning: Timeout waiting for namespace deletion after {max_attempts * poll_interval} seconds")
    
    print("Starting Redshift environment cleanup...")
    print("=" * 60)
    
    # 1. Delete Redshift workgroup first
    print(f"Step 1: Deleting Redshift workgroup {REDSHIFT_WORKGROUP}")
    try:
        redshift_client.delete_workgroup(workgroupName=REDSHIFT_WORKGROUP)
        print("  Workgroup deletion initiated")
        wait_for_workgroup_deleted(REDSHIFT_WORKGROUP)
    except redshift_client.exceptions.ResourceNotFoundException:
        print("  Workgroup already deleted or does not exist")
    except Exception as e:
        print(f"  Error deleting workgroup: {str(e)}")
    
    # 2. Delete Redshift namespace
    print(f"\nStep 2: Deleting Redshift namespace {REDSHIFT_NAMESPACE}")
    try:
        redshift_client.delete_namespace(namespaceName=REDSHIFT_NAMESPACE)
        print("  Namespace deletion initiated")
        wait_for_namespace_deleted(REDSHIFT_NAMESPACE)
    except redshift_client.exceptions.ResourceNotFoundException:
        print("  Namespace already deleted or does not exist")
    except Exception as e:
        print(f"  Error deleting namespace: {str(e)}")
    
    # 3. Empty and delete S3 bucket
    print(f"\nStep 3: Deleting S3 bucket {S3_BUCKET}")
    try:
        bucket = s3.Bucket(S3_BUCKET)
        
        # Check if bucket exists
        s3_client.head_bucket(Bucket=S3_BUCKET)
        
        # Delete all objects in the bucket
        print("  Emptying bucket contents...")
        objects_to_delete = []
        for obj in bucket.objects.all():
            objects_to_delete.append({'Key': obj.key})
        
        if objects_to_delete:
            bucket.delete_objects(Delete={'Objects': objects_to_delete})
            print(f"    Deleted {len(objects_to_delete)} objects")
        else:
            print("    Bucket was already empty")
        
        # Delete the bucket
        print("  Deleting bucket...")
        bucket.delete()
        print("  S3 bucket deleted successfully")
        
    except s3_client.exceptions.NoSuchBucket:
        print("  S3 bucket already deleted or does not exist")
    except Exception as e:
        print(f"  Error deleting S3 bucket: {str(e)}")
    
    # 4. Delete Secrets Manager secret
    print(f"\nStep 4: Deleting Secrets Manager secret {secret_name}")
    try:
        secrets_client.delete_secret(
            SecretId=secret_name,
            ForceDeleteWithoutRecovery=True  # Skip recovery window for cleanup
        )
        print("  Secrets Manager secret deleted successfully")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("  Secret already deleted or does not exist")
    except Exception as e:
        print(f"  Error deleting secret: {str(e)}")
    
    # 5. Delete IAM role and policies
    print(f"\nStep 5: Deleting IAM role {redshift_role_arn.split('/')[-1]}")
    role_name = redshift_role_arn.split('/')[-1]
    try:
        # Check if role exists
        iam_client.get_role(RoleName=role_name)
        
        # Detach managed policies
        print("  Detaching managed policies...")
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
        for policy in attached_policies:
            policy_arn = policy['PolicyArn']
            iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
            print(f"    Detached policy: {policy['PolicyName']}")
            
            # Delete custom policies (not AWS managed)
            if not policy_arn.startswith('arn:aws:iam::aws:policy/'):
                try:
                    iam_client.delete_policy(PolicyArn=policy_arn)
                    print(f"    Deleted custom policy: {policy['PolicyName']}")
                except Exception as e:
                    print(f"    Could not delete policy {policy['PolicyName']}: {str(e)}")
        
        # Delete inline policies
        print("  Deleting inline policies...")
        inline_policies = iam_client.list_role_policies(RoleName=role_name)['PolicyNames']
        for policy_name in inline_policies:
            iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
            print(f"    Deleted inline policy: {policy_name}")
        
        # Delete the role
        iam_client.delete_role(RoleName=role_name)
        print("  IAM role deleted successfully")
        
    except iam_client.exceptions.NoSuchEntityException:
        print("  IAM role already deleted or does not exist")
    except Exception as e:
        print(f"  Error deleting IAM role: {str(e)}")
    
    print("\n" + "=" * 60)
    print("Redshift environment cleanup completed")
    print("\nSummary of deleted resources:")
    print(f"  - Redshift Workgroup: {REDSHIFT_WORKGROUP}")
    print(f"  - Redshift Namespace: {REDSHIFT_NAMESPACE}")
    print(f"  - S3 Bucket: {S3_BUCKET}")
    print(f"  - Secrets Manager Secret: {secret_name}")
    print(f"  - IAM Role: {role_name}")

# Usage:
cleanup_redshift_environment()

Starting Redshift environment cleanup...
Step 1: Deleting Redshift workgroup sds-ecommerce-wg-1093701
  Workgroup deletion initiated
  Waiting for workgroup sds-ecommerce-wg-1093701 to be deleted...
    Workgroup status: DELETING
    Workgroup status: DELETING
    Workgroup status: DELETING
    Workgroup status: DELETING
    Workgroup status: DELETING
    Workgroup status: DELETING
    Workgroup status: DELETING
    Workgroup deleted successfully

Step 2: Deleting Redshift namespace sds-ecommerce-1093701
  Namespace deletion initiated
  Waiting for namespace sds-ecommerce-1093701 to be deleted...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Namespace still exists, waiting...
    Na